In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.

city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,noumea,NC,22-10-29 00:02:12,-22.2763,166.4572,84.18,71,9,7.05
1,1,taos,US,22-10-29 00:04:02,36.4072,-105.5731,54.00,26,0,0.00
2,2,paamiut,GL,22-10-29 00:02:59,61.9940,-49.6678,33.93,85,100,23.13
3,3,houma,US,22-10-28 23:59:48,29.5958,-90.7195,68.00,93,0,11.50
4,4,ponta do sol,PT,22-10-29 00:04:02,32.6667,-17.1000,72.01,82,16,5.53


In [4]:
# Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [7]:
# Create a Maximum Temperature Heatmap
# Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Assign the locations to an array of latitude and longitude pairs.
locations = city_data_df[["Lat", "Lng"]]

# 2. Get the maximum temperature
max_temp = city_data_df["Max Temp"]

# 3. Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure()

# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
heatmap_layer = gmaps.heatmap_layer(locations, weights=temperatures)

# 5. Add the heatmap layer.
fig.add_layer(heatmap_layer)

# 6. Call the figure to plot the data.
fig

NameError: name 'temperatures' is not defined